In [2]:
import tensorflow as tf
import mlflow
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import os
import numpy as np
import mlflow
from PIL import Image
import cv2

In [3]:
import splitfolders

input_folder = 'data/'
splitfolders.ratio(input_folder, output="output_folder", seed=42, ratio=(.7, .2, .1), group_prefix=None)  

Copying files: 0 files [00:00, ? files/s]

Copying files: 400 files [00:00, 748.41 files/s]


In [4]:
# Path to your folder containing images
train_path="output_folder/train"
test_path="output_folder/test"
val_path="output_folder/val"

In [5]:
# Initialize an empty list to hold the image arrays
x_train = []

# Iterate over all image files in the folder
for filename in os.listdir(train_path):
    sub_path=train_path+"/"+ filename
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)

# Convert the list of arrays into a NumPy array
x_train_np = np.array(x_train)

# Print the shape of the resulting array (should be something like (num_images, 256, 256, 3))
print("Shape of the NumPy array:", x_train_np.shape)

Shape of the NumPy array: (280, 224, 224, 3)


In [6]:
x_test=[]
for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)

x_val=[]
for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)

# Convert the list of arrays into a NumPy array
x_test_np = np.array(x_test)
x_val_np = np.array(x_val)
# Print the shape of the resulting array (should be something like (num_images, 256, 256, 3))
print("Shape of the NumPy array:", x_test_np.shape)
print("Shape of the NumPy array:", x_val_np.shape)

Shape of the NumPy array: (40, 224, 224, 3)
Shape of the NumPy array: (80, 224, 224, 3)


In [7]:
#Normalizing the data
train_x=x_train_np/255.0 
test_x=x_test_np/255.0 
val_x=x_val_np/255.0

In [8]:
# ImageDataGenerator for data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1.0/255)
print(train_datagen)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [9]:
# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    shuffle=True,
    class_mode='categorical'  
)

# Load and preprocess the validation data
validation_generator = validation_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    shuffle=False,
    class_mode='categorical'
)

print(train_datagen)
print(train_generator)

Found 280 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [10]:
#Build a model
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(224, 224,3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation = 'softmax')
    ])

c:\Users\Divyaprabha\.conda\envs\ml-pipeline\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 150528)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    19,267,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,267,970 (73.50 MB)

 Trainable params: 19,267,970 (73.50 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(train_generator, 
          steps_per_epoch = train_generator.samples // 2,
          epochs=20, 
         validation_data=validation_generator,
         validation_steps=validation_generator.samples // 2,
         verbose=1)

model.save('my_model.keras')

Epoch 1/20


c:\Users\Divyaprabha\.conda\envs\ml-pipeline\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  9/140 ━━━━━━━━━━━━━━━━━━━━ 25s 194ms/step - accuracy: 0.5594 - loss: 50.6766

c:\Users\Divyaprabha\.conda\envs\ml-pipeline\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5640 - loss: 39.4557 - val_accuracy: 0.6750 - val_loss: 1.3918
Epoch 2/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5620 - loss: 2.6455 - val_accuracy: 0.4875 - val_loss: 2.6452
Epoch 3/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6916 - loss: 1.1833 - val_accuracy: 0.6500 - val_loss: 1.4340
Epoch 4/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6945 - loss: 1.2195 - val_accuracy: 0.6125 - val_loss: 1.4815
Epoch 5/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8299 - loss: 0.5716 - val_accuracy: 0.7500 - val_loss: 0.8523
Epoch 6/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8374 - loss: 0.4846 - val_accuracy: 0.7375 - val_loss: 0.7691
Epoch 7/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8928 - loss: 0.2948 - val_accuracy: 0.7875 - val_loss: 0.9246
Epoch 8/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8803 - loss: 0.3128 - val_accuracy: 0.77

In [13]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.7750 - loss: 0.6374
Validation Accuracy: 80.00%
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 1.0000 - loss: 0.0474
Train Accuracy: 100.00%


In [14]:
# Prediction using trained model
print(type(test_x))
print(test_x.shape)

<class 'numpy.ndarray'>
(40, 224, 224, 3)


In [15]:
#Batch Prediction
predictions = model.predict(test_x)

print(predictions)
# If you want to get the predicted class for each image
predicted_classes = np.argmax(predictions, axis=1)

# Print predicted classes
print("Predicted classes:", predicted_classes)



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
[[9.8973364e-01 1.0266399e-02]
 [9.9996376e-01 3.6241017e-05]
 [9.3819886e-01 6.1801068e-02]
 [9.6005243e-01 3.9947558e-02]
 [4.4293955e-01 5.5706042e-01]
 [2.9213564e-02 9.7078639e-01]
 [9.9984109e-01 1.5883555e-04]
 [9.9992919e-01 7.0773844e-05]
 [2.0191494e-01 7.9808503e-01]
 [9.9948597e-01 5.1399320e-04]
 [1.2322666e-01 8.7677336e-01]
 [2.0504861e-01 7.9495132e-01]
 [9.9999905e-01 9.4997353e-07]
 [8.6959201e-01 1.3040800e-01]
 [9.9325037e-01 6.7496160e-03]
 [2.2203256e-01 7.7796739e-01]
 [7.8922200e-01 2.1077806e-01]
 [9.9129069e-01 8.7093255e-03]
 [9.1623414e-01 8.3765879e-02]
 [9.5761633e-01 4.2383693e-02]
 [8.7038565e-01 1.2961431e-01]
 [8.3968267e-03 9.9160320e-01]
 [9.7663358e-02 9.0233666e-01]
 [4.5239949e-03 9.9547607e-01]
 [9.4329447e-01 5.6705557e-02]
 [4.8888388e-01 5.1111609e-01]
 [9.8745036e-01 1.2549611e-02]
 [6.8026745e-01 3.1973258e-01]
 [9.1398247e-02 9.0860170e-01]
 [8.9574300e-02 9.1042572e-01]
 [3.9545882e-01 6.0454118e-01]
 

In [139]:
#Single image Prediction
pred_path = '20.jpg'
img = image.load_img(pred_path, target_size=(224,224))

img_array = image.img_to_array(img)
img_array = img_array / 255.0 

img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
print(predictions) 

predicted_class = np.argmax(predictions, axis=1)
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
[[6.035186e-13 1.000000e+00]]
Predicted class: [1]


In [ ]:
from mlflow.models.signature import infer_signature
model_signature = infer_signature(test_x, model.predict(test_x))
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
from datetime import datetime
input_data_path = 'v_data'
exp_timestamp = datetime.now().strftime("%Y%m%d")
exp_name = "custom_ds" + exp_timestamp
print(exp_name)

run_timestamp = datetime.now().strftime("%Y%m%d--%H%M%S")
run_name = "custom_ds_run"+ run_timestamp
print(run_name)

#mlflow.tensorflow.autolog(log_every_epoch=2)

mlflow.set_experiment(exp_name)
with mlflow.start_run(run_name=run_name):
        #mlflow.log_artifact(input_data_path, artifact_path="input_data")
        mlflow.log_param("batch_size", 2)
        #mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", 20)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("val_loss", val_loss)
        mlflow.log_metric("val_accuracy", val_accuracy)
        
        mlflow.tensorflow.log_model(model, "custom_ds", signature=model_signature,registered_model_name='custom_ds_model')
mlflow.end_run()
